In [62]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.cross_validation import KFold, cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import numpy as np
import time
import datetime
import pandas as pd
from sklearn.preprocessing import scale
import tensorflow as tf

features = pd.read_csv('train/train_big.csv', header = 0)
X = features.drop(['order_id','cancel'], axis=1)
y = features['cancel']
print('{0} train samples loaded'.format(len(X)))

#print(X.head())

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

print(type(X))

374200 train samples loaded
<class 'pandas.core.frame.DataFrame'>


In [63]:
#X.dtypes

for c in X.columns:
    if X[c].dtype == np.float64 or X[c].dtype == np.int64:
        print(X[c].dtype)
        X[c] = X[c].astype('float32')
        
print(X.dtypes)

int64
int64
int64
int64
int64
float64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
int64
float64
float64
int64
int64
int64
float64
int64
int64
float64
int64
int64
float64
int64
int64
float64
float64
rooms                   float32
bathrooms               float32
total_cents             float32
subtotal_cents          float32
discount                float32
discount_prc            float32
payment_type             object
creation_mean            object
dow                     float32
hot                     float32
refrigerator_flg        float32
oven_flg                float32
windows_fls             float32
ironing_flg             float32
kitchen_cabinets_flg    float32
tableware_flg           float32
balconies_flg           float32
microwave_flg           float32
keys_delivery_flg       float32
consumables_flg         float32
shower_flg              float32
bedlinen_flg            float32
cat_flg   

In [64]:
def fill_na(df, col_name):
    df[col_name] = df[col_name].fillna(df[col_name].mean())
    df = df.fillna(0)
    return df

def le_fit_transform(df, col_name):
    le = LabelEncoder()
    le.fit(df[col_name])
    df[col_name] = le.transform(df[col_name])
    return df, le

def le_transform(df, col_name, le):
    df[col_name] = le.transform(df[col_name])
    return df

def ohc_fit_transform(df, col_name):
    ohe = OneHotEncoder(sparse=False)
    ohe.fit(df[col_name].values.reshape(-1, 1))
    encoded_col = ohe.transform(df[col_name].values.reshape(-1, 1))
    #encoded_col = ohe.fit_transform(df[col_name].values.reshape(-1, 1))
    tmp = pd.DataFrame(encoded_col, columns=[col_name + str(i) for i in range(encoded_col.shape[1])], index = df.index)
    df = pd.concat([df, tmp], axis = 1)
    df = df.drop([col_name], axis = 1)
    return df, ohe

def ohc_transform(df, col_name, ohe):
    encoded_col = ohe.transform(df[col_name].values.reshape(-1, 1))
    tmp = pd.DataFrame(encoded_col, columns=[col_name + str(i) for i in range(encoded_col.shape[1])], index = df.index)
    df = pd.concat([df, tmp], axis = 1)
    df = df.drop([col_name], axis = 1)
    return df

def exponentiation(df, cols):
    for c in cols:
        df[c + '_sq'] = df[c]**2
        df[c + '_sqrt'] = np.sqrt(df[c])
    return df
        

 
X = fill_na(X, 'shifts_num')

X, ohc_dow = ohc_fit_transform(X, 'dow')

X, le_type = le_fit_transform(X, 'type')
X, ohc_type = ohc_fit_transform(X, 'type')

X, le_creation_mean = le_fit_transform(X, 'creation_mean')
X, ohc_creation_mean = ohc_fit_transform(X, 'creation_mean')

X, le_payment_type = le_fit_transform(X, 'payment_type')
X, ohc_payment_type = ohc_fit_transform(X, 'payment_type')

X = exponentiation(
    X, ['paid_all','canceled_all','paid_16d','canceled_16d', 'paid_30d','canceled_30d', 'paid_60d','canceled_60d'])

X = scale(X)

train_dataset = X[0:250000]
valid_dataset = X[250001:320000]
test_dataset = X[320001:]

train_labels = y[0:250000]
valid_labels = y[250001:320000]
valid_labels = y[320001:]

/Users/gapon/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/data.py:167: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


In [69]:
X

array([[ 0.14441142, -0.30956713,  0.9302052 , ..., -1.210928  ,
         1.72788389,  2.16149745],
       [ 2.40554172,  2.78916817,  2.16847955, ..., -1.210928  ,
         4.62431578,  2.97548152],
       [-2.11671888, -0.30956713, -0.95447107, ..., -0.10303591,
        -0.2030707 ,  0.16467537],
       ..., 
       [-2.11671888, -0.30956713, -2.35292586, ..., -0.10303591,
        -0.2030707 ,  0.16467537],
       [ 0.14441142, -0.30956713,  1.37325749, ..., -1.210928  ,
        -0.2030707 ,  0.16467537],
       [-0.98615373, -0.30956713, -0.44325689, ..., -0.10303591,
        -0.2030707 ,  0.16467537]])

In [65]:
batch_size = 64
fc1_size = 1024
fc2_size = 512
fc3_size = 32
num_labels = 1

input_size = X.shape[1]

print(input_size)

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, input_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    tf_beta = tf.placeholder(tf.float32)
    global_step = tf.Variable(0)  # count the number of steps taken.

    # Variables.
    # stddev is very important!!!
    W1 = tf.Variable(
        tf.truncated_normal([input_size, fc1_size], stddev=np.sqrt(2.0 / input_size)))
    b1 = tf.Variable(tf.zeros([fc1_size]))

    W2 = tf.Variable(tf.truncated_normal([fc1_size, fc2_size], stddev=np.sqrt(2.0 / fc1_size)))
    b2 = tf.Variable(tf.zeros([fc2_size]))

    W3 = tf.Variable(tf.truncated_normal([fc2_size, fc3_size], stddev=np.sqrt(2.0 / fc2_size)))
    b3 = tf.Variable(tf.zeros([fc3_size]))

    W4 = tf.Variable(tf.truncated_normal([fc3_size, num_labels], stddev=np.sqrt(2.0 / fc3_size)))
    b4 = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    
    y1 = tf.nn.relu(tf.matmul(tf_train_dataset, W1) + b1)
    # y1 = tf.nn.dropout(y1, 0.5)

    y2 = tf.nn.relu(tf.matmul(y1, W2) + b2)
    # y2 = tf.nn.dropout(y2, 0.5)

    y3 = tf.nn.relu(tf.matmul(y2, W3) + b3)
    # y3 = tf.nn.dropout(y3, 0.5)

    logits = tf.matmul(y3, W4) + b4
    
    output = tf.sigmoid(logits)

    loss = tf.reduce_sum(tf.square(output - tf_train_labels))

    loss = loss + tf_beta * (tf.nn.l2_loss(W1) + tf.nn.l2_loss(b1) + tf.nn.l2_loss(W2) + tf.nn.l2_loss(b2) +
                             tf.nn.l2_loss(W3) + tf.nn.l2_loss(b3) + tf.nn.l2_loss(W4) + tf.nn.l2_loss(b4))

    # Optimizer
    learning_rate = tf.train.exponential_decay(0.5, global_step, 1000, 0.7, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.sigmoid(logits)

    y1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, W1) + b1)
    y2_valid = tf.nn.relu(tf.matmul(y1_valid, W2) + b2)
    y3_valid = tf.nn.relu(tf.matmul(y2_valid, W3) + b3)
    valid_logits = tf.matmul(y3_valid, W4) + b4
    valid_prediction = tf.sigmoid(valid_logits)

    y1_test = tf.nn.relu(tf.matmul(tf_test_dataset, W1) + b1)
    y2_test = tf.nn.relu(tf.matmul(y1_test, W2) + b2)
    y3_test = tf.nn.relu(tf.matmul(y2_test, W3) + b3)
    test_logits = tf.matmul(y3_test, W4) + b4
    test_prediction = tf.sigmoid(test_logits)

82


TypeError: Input 'b' of 'MatMul' Op has type float32 that does not match type float64 of argument 'a'.

In [44]:
# Let's run it:
num_steps = 12001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (len(train_labels) - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size)]
        #print(batch_data.shape)
        batch_labels = train_labels[offset:(offset + batch_size)].reshape(64,1)
        #print(batch_labels.shape)
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels, tf_beta: 0.001438}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            # print("Minibatch accuracy: %.1f%%" % roc_auc_score(batch_labels, predictions))
            print(valid_prediction.eval())
            #print("Validation accuracy:", roc_auc_score(valid_labels, valid_prediction.eval()))
    print("Final Test accuracy: %.1f%%" % roc_auc_score( test_labels, test_prediction.eval()))

Initialized
Minibatch loss at step 0: 24.135878


/Users/gapon/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:14: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead


[[ 0.]
 [ 0.]
 [ 0.]
 ..., 
 [ 0.]
 [ 0.]
 [ 0.]]
Minibatch loss at step 500: 27.092171
[[ 0.]
 [ 0.]
 [ 0.]
 ..., 
 [ 0.]
 [ 0.]
 [ 0.]]
Minibatch loss at step 1000: 39.019123
[[ 0.]
 [ 0.]
 [ 0.]
 ..., 
 [ 0.]
 [ 0.]
 [ 0.]]
Minibatch loss at step 1500: 25.616014
[[ 0.]
 [ 0.]
 [ 0.]
 ..., 
 [ 0.]
 [ 0.]
 [ 0.]]
Minibatch loss at step 2000: 30.372355
[[  4.15141680e-24]
 [  6.09493869e-19]
 [  9.02427889e-19]
 ..., 
 [  3.57051067e-23]
 [  0.00000000e+00]
 [  0.00000000e+00]]
Minibatch loss at step 2500: 41.261772
[[  1.58920761e-12]
 [  5.68557867e-10]
 [  6.90280166e-10]
 ..., 
 [  4.60141083e-12]
 [  4.88991171e-21]
 [  2.04730749e-31]]
Minibatch loss at step 3000: 36.704903
[[ 0.]
 [ 0.]
 [ 0.]
 ..., 
 [ 0.]
 [ 0.]
 [ 0.]]


KeyboardInterrupt: 